In [1]:
from htm.bindings.sdr import SDR, Metrics
from htm.algorithms import SpatialPooler
from htm.bindings.algorithms import TemporalMemory
from htm.algorithms.anomaly_likelihood import AnomalyLikelihood
import numpy as np
import pandas as pd
import pathlib
import datetime
import csv
from datetime import datetime
import os
from htm.encoders.rdse import RDSE, RDSE_Parameters
import time
import traceback
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn_extra.cluster import KMedoids
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from random import shuffle


In [2]:
inputSources = [
   "monthly_sp500_pca.csv",
   "weekly_dow_jones.csv",
   "weekly_nasdaq.csv",
   "weekly_sp500.csv",
   "monthly_vix_close.csv",
   "monthly_vix_high.csv",
   "monthly_vix_low.csv",
   "monthly_vix_open.csv",
   "daily_natural_gas.csv",
   "daily_oil_prices.csv",
   "value1_vix_close.csv",
   "value1_vix_high.csv",
   "value1_vix_low.csv",
   "value1_vix_open.csv",
   "monthly_gold_prices.csv"
]

In [3]:

config = {
    'enc': {
        "value" :
            {'resolution': 0.88, 'size': 700, 'sparsity': 0.02},
        "time": 
            {'timeOfDay': (30, 1), 'weekend': 21}
    },
    'sp': {
        'inputDimensions': None,
        'columnDimensions': (1638,),
        'potentialPct': 0.85,
        'potentialRadius': None,
        'globalInhibition': True,
        'localAreaDensity': 0.04395604395604396,
        'synPermInactiveDec': 0.006,
        'synPermActiveInc': 0.04,
        'synPermConnected': 0.13999999999999999,
        'boostStrength': 3.0,
        'wrapAround': True,
        'seed': 1,
        'learn': False,
    },
    'tm': {
        'cellsPerColumn': 13,
        'activationThreshold': 17,
        'initialPermanence': 0.21,
        'minThreshold': 10,
        'maxNewSynapseCount': 32,
        'permanenceIncrement': 0.1,
        'permanenceDecrement': 0.1,
        'predictedSegmentDecrement': 0.0,
        'maxSegmentsPerCell': 128,
        'maxSynapsesPerSegment': 64,
        'learn': True
    },
    'anomaly': {'period': 1000},
    'learnRows': 100,
    'reflexSize': 2048,
    'accuracyThreshold': 0.5,
    'controlThreshold': 0.7
}


In [4]:
class ReflexiveMemory:
  def __init__(self, reflexSize, dimensions_dense, dimensions_sparse_sp):
    self.acKey0 = None
    self.pairs = {}
    self.tableSize  = reflexSize
    self.dimensions_dense = dimensions_dense
    self.dimensions_sparse_sp = dimensions_sparse_sp

  def add(self, denseColumns):
    acKey1 = '-'.join(map(str, denseColumns.sparse))
    if(self.acKey0 != None):

      sequence = self.pairs.get(self.acKey0, {})
      sequence_data = sequence.get(acKey1, {
         "count": 0,
         "time": datetime.now()
      })
      sequence_data["count"] = sequence_data["count"] + 1
      sequence_data["time"] = datetime.now()

      if self.pairs.get(self.acKey0, None) is None:
        self.pairs[self.acKey0] = { acKey1: sequence_data }
      else:
        self.pairs[self.acKey0][acKey1] = sequence_data
        
      table_entries = 0
      oldKey1 = None
      oldKey2 = None
      oldTime = datetime.now()
      for key1, value1 in self.pairs.items():
        table_entries = table_entries + len(value1.items())
        for key2, value2 in value1.items():
          if value2['time'] < oldTime:
            oldKey1 = key1
            oldKey2 = key2
            oldTime = value2['time']
      if table_entries > self.tableSize:
        del self.pairs[oldKey1][oldKey2]
        if len(self.pairs[oldKey1].items()) == 0:
          del self.pairs[oldKey1]

    self.acKey0 = acKey1

  def predict(self, denseColumns):
    return_count = 0
    return_sdr = None

    acKey = '-'.join(map(str, denseColumns.sparse))
    sequences = self.pairs.get(acKey, {})
    for sequence_key, sequence_data in sequences.items():
      if sequence_data["count"] > return_count:
        return_count = sequence_data["count"]
        return_sdr = sequence_key

    if return_sdr is not None:
      tmp_sdr = SDR( self.dimensions_dense )
      tmp_sdr.sparse = list(map(int, return_sdr.split('-')))
      return_sdr = tmp_sdr
    else:
      return_count = None

    return return_count, return_sdr


In [5]:
class ControlUnit:
  def __init__(self, controlThreshold):
    self.anomalyRM = []
    self.anomalyTM = []
    self.anomalyNU = []
    self.anomalyCU = []
    self.historyRM = []
    self.historyTM = []
    self.historyGT = []
    self.historyCU = []
    self.countRMCU = 0
    self.controlThreshold = controlThreshold

  def anomalyScore(self, y, x):
      if np.count_nonzero(y) != 0:
          return 1 - np.count_nonzero((x & y)) / np.count_nonzero(y)
      return 1

  def compute(self, denseColumns1, sp, tm, rm):

    if rm.acKey0 is not None:

      denseColumns0 = SDR( rm.dimensions_dense )
      denseColumns0.sparse = list(map(int, rm.acKey0.split('-')))

      tm.activateDendrites(True)
      predictiveCells = tm.getPredictiveCells()

      predictiveColumns = SDR( rm.dimensions_sparse_sp )
      predictiveColumns.sparse = list(set(sorted(list(np.where(predictiveCells.dense == 1)[0]))))

      reflexiveColumns = SDR( rm.dimensions_sparse_sp )
      reflexiveCount, denseReflexiveColumns = rm.predict(denseColumns0)
      if denseReflexiveColumns is not None:
        sp.compute(denseReflexiveColumns, False, reflexiveColumns)

      activeColumns0 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns0, False, activeColumns0)

      activeColumns1 = SDR( rm.dimensions_sparse_sp )
      sp.compute(denseColumns1, False, activeColumns1)

      self.historyRM.append( reflexiveColumns.dense )
      self.historyTM.append( predictiveColumns.dense )
      self.historyGT.append( activeColumns1.dense )

      self.anomalyNU.append(tm.anomaly)
      self.anomalyRM.append( self.anomalyScore(activeColumns1.dense, reflexiveColumns.dense) )
      self.anomalyTM.append( self.anomalyScore(activeColumns1.dense, predictiveColumns.dense) )

      #  = sum(self.anomalyRM[-8:])/len(self.anomalyRM[-8:])
      if self.anomalyRM[-1] < self.anomalyTM[-1]:
        self.anomalyCU.append( self.anomalyRM[-1] )
        self.historyCU.append( self.historyRM[-1] )
        self.countRMCU = self.countRMCU + 1
      else:
        self.anomalyCU.append( self.anomalyTM[-1] )
        self.historyCU.append( self.historyTM[-1] )

#      if overlapRM < (1 - 0.5):
#        oldKey1 = '-'.join(map(str, denseColumns0.sparse))
#        oldKey2 = '-'.join(map(str, denseReflexiveColumns.sparse))
#        if reflexiveCount > 1:
#          self.pairs[oldKey1][oldKey2]["count"] = reflexiveCount - 1
#        else:
#          del self.pairs[oldKey1][oldKey2]
#          if len(self.pairs[oldKey1].items()) == 0:
#            del self.pairs[oldKey1]
  

In [6]:
input_path = pathlib.Path('../datasets/numenta')
dataset_metrics = []

pbar = tqdm(total=len(inputSources))
for dataset in inputSources:

    tm_infer_tm = 0
    tm_infer_rm = 0

    records = []
    with open(input_path.joinpath(dataset), "r") as fin:
        reader = csv.reader(fin)
        headers = next(reader)
        next(reader)
        next(reader)
        for record in reader:
            records.append(record)
        
    scalarEncoderParams = RDSE_Parameters()
    scalarEncoderParams.size = config["enc"]["value"]["size"]
    scalarEncoderParams.sparsity = config["enc"]["value"]["sparsity"]
    scalarEncoderParams.resolution = config["enc"]["value"]["resolution"]
    scalarEncoder = RDSE( scalarEncoderParams )
    encodingWidth = (scalarEncoder.size)

    config['sp']['inputDimensions'] = (encodingWidth,)
    config['sp']['potentialRadius'] = encodingWidth

    sp = SpatialPooler(
        inputDimensions = config['sp']['inputDimensions'],
        columnDimensions = config['sp']['columnDimensions'],
        potentialPct = config['sp']['potentialPct'],
        potentialRadius = config['sp']['potentialRadius'],
        globalInhibition = config['sp']['globalInhibition'],
        localAreaDensity = config['sp']['localAreaDensity'],
        synPermInactiveDec = config['sp']['synPermInactiveDec'],
        synPermActiveInc = config['sp']['synPermActiveInc'],
        synPermConnected = config['sp']['synPermConnected'],
        boostStrength = config['sp']['boostStrength'],
        wrapAround = config['sp']['wrapAround'],
        seed = config['sp']['seed']
    )

    tm = TemporalMemory(
        columnDimensions = config['sp']['columnDimensions'],
        cellsPerColumn = config['tm']['cellsPerColumn'],
        activationThreshold = config['tm']['activationThreshold'],
        initialPermanence = config['tm']['initialPermanence'],
        connectedPermanence = config['sp']['synPermConnected'],
        minThreshold = config['tm']['minThreshold'],
        maxNewSynapseCount = config['tm']['maxNewSynapseCount'],
        permanenceIncrement = config['tm']['permanenceIncrement'],
        permanenceDecrement = config['tm']['permanenceDecrement'],
        predictedSegmentDecrement = config['tm']['predictedSegmentDecrement'],
        maxSegmentsPerCell = config['tm']['maxSegmentsPerCell'],
        maxSynapsesPerSegment = config['tm']['maxSynapsesPerSegment']
    )

    rm = ReflexiveMemory( config['reflexSize'] , config['sp']['inputDimensions'], config['sp']['columnDimensions'])
    cu = ControlUnit( config['controlThreshold'] )

    try:
        
        for count, record in enumerate(records):
            
            learn_sp = config['sp']['learn']
            learn_tm = config['tm']['learn']
            if count < config['learnRows']:
                learn_sp = True
                learn_tm = True

            consumption = float(record[1])
            consumptionBits = scalarEncoder.encode(consumption)
            encoding = SDR( consumptionBits )

            cu.compute(encoding, sp, tm, rm)

            rm.add(encoding)

            activeColumns = SDR( sp.getColumnDimensions() )

            tmp_tm = time.time()
            sp.compute(encoding, learn_sp, activeColumns)
            tm.compute(activeColumns, learn=learn_tm)
            tm_infer_tm = tm_infer_tm + (time.time() - tmp_tm)

            tmp_tm = time.time()
            rm.predict(encoding)
            tm_infer_rm = tm_infer_rm + (time.time() - tmp_tm)

    except Exception as e:
        print(traceback.format_exc())
        print(e)

    def roc_auc_score_multiclass(y_true, y_pred):
        scores = []
        for y_class in set(y_true):
            y_true_class = [True if x == y_class else False for x in y_true]
            y_pred_class = [True if x == y_class else False for x in y_pred]
            scores.append(roc_auc_score(y_true_class, y_pred_class))
        return sum(scores) / len(scores)

    def match(y, x, idx1, cu, accuracyThreshold):
        n_samples = len(y)
        score1 = cu.anomalyScore(y[idx1], x[idx1])
        if score1 > (1 - accuracyThreshold):
            idx_closest = None
            score_closest = None
            for idx2 in range(n_samples):
                score2 = cu.anomalyScore(y[idx2], x[idx1])
                if score_closest is None or score_closest > score2:
                    score_closest = score2
                    idx_closest = idx2
            return idx_closest
        return idx1

    def calculateMetricsAnomaly(anomaly_scores, config, suffix):

        metric = {}

        anomaly_probability = []
        anomaly_history = AnomalyLikelihood(config["anomaly"]["period"])
        for anomaly_value in anomaly_scores:
            anomaly_probability.append( anomaly_history.compute(anomaly_value) )

        metric['anomaly-avg-'+suffix] = sum(anomaly_scores) / len(anomaly_scores)
        metric['anomaly-samples-'+suffix] = len(anomaly_scores)
        metric['anomaly-prob-avg-'+suffix] = np.count_nonzero(anomaly_probability) / len(anomaly_probability)

        return metric

    def calculateMetrics(Y_dataset, X_dataset, anomaly_scores, total_infe_time, config, cu, suffix):

        metric = {}

        n_samples = len(Y_dataset)

        Y_labels = list(range(n_samples))
        X_labels = [ match(Y_dataset, X_dataset, idx, cu, config['accuracyThreshold']) for idx in range(n_samples)]
        precision, recall, fscore, support = precision_recall_fscore_support(Y_labels, X_labels, average='macro', zero_division=0.0)
        metric['total-infer-time-'+suffix] = total_infe_time
        metric['infer-time-'+suffix] = total_infe_time / len(anomaly_scores)
        metric['accuracy-'+suffix] = accuracy_score(Y_labels, X_labels)
        metric['precision-'+suffix] = precision
        metric['recall-'+suffix] = recall
        metric['fscore-'+suffix] = fscore
        metric['support-'+suffix] = support
        metric['auc-'+suffix] = roc_auc_score_multiclass(Y_labels, X_labels)

        metric.update( calculateMetricsAnomaly(anomaly_scores, config, suffix) )

        return metric
    
    tm_infer_cu = 0
    avg_infer_time_rm = tm_infer_rm / len(cu.anomalyRM)
    avg_infer_time_tm = tm_infer_tm / len(cu.anomalyTM)
    tm_infer_cu = tm_infer_cu + (avg_infer_time_rm * cu.countRMCU)
    tm_infer_cu = tm_infer_cu + (avg_infer_time_tm * (len(cu.anomalyCU) - cu.countRMCU))

    metric = {}
    metric['dataset'] = dataset
    metric['cu-rm-count'] = cu.countRMCU
    metric.update( calculateMetrics(cu.historyGT, cu.historyRM, cu.anomalyRM, tm_infer_rm, config, cu, 'rm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyTM, cu.anomalyTM, tm_infer_tm, config, cu, 'tm') )
    metric.update( calculateMetrics(cu.historyGT, cu.historyCU, cu.anomalyCU, tm_infer_cu, config, cu, 'cu') )
    metric.update( calculateMetricsAnomaly(cu.anomalyNU, config, 'nupic') )

    dataset_metrics.append(metric)
    pbar.update(1)
    # break

pbar.close()

100%|██████████| 15/15 [05:36<00:00, 22.41s/it]


In [7]:
df = pd.DataFrame(dataset_metrics)
df.to_csv('metrics.csv', index=False)

In [8]:
table_1_features = ['dataset','accuracy-cu','accuracy-rm','accuracy-tm','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic']
df[table_1_features]

,dataset,accuracy-cu,accuracy-rm,accuracy-tm,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic
0,monthly_sp500_pca.csv,0.436319,0.169409,0.411335,0.568590,0.807748,0.585449,0.585449
1,weekly_dow_jones.csv,0.037037,0.008658,0.034632,0.933315,0.984454,0.935132,0.935132
2,weekly_nasdaq.csv,0.222115,0.074038,0.216827,0.754400,0.906177,0.759195,0.759235
3,weekly_sp500.csv,0.362326,0.137434,0.356559,0.613761,0.843591,0.621082,0.621109
4,monthly_vix_close.csv,0.976049,0.743210,0.969630,0.047414,0.327473,0.054763,0.055010
5,monthly_vix_high.csv,0.960247,0.679506,0.945185,0.063995,0.356163,0.078793,0.079040
6,monthly_vix_low.csv,0.979259,0.768889,0.976543,0.031557,0.298951,0.036578,0.036824
7,monthly_vix_open.csv,0.961235,0.707407,0.950617,0.055960,0.355967,0.068316,0.068532
8,daily_natural_gas.csv,0.996378,0.955674,0.995516,0.005426,0.065698,0.008657,0.008830
9,daily_oil_prices.csv,0.994940,0.921687,0.993976,0.016821,0.210119,0.019428,0.019533


In [9]:
table_2_features = ['dataset','anomaly-avg-cu','anomaly-avg-rm','anomaly-avg-tm','anomaly-avg-nupic','anomaly-prob-avg-cu','anomaly-prob-avg-rm','anomaly-prob-avg-tm','anomaly-prob-avg-nupic','anomaly-samples-cu','anomaly-samples-rm','anomaly-samples-tm','anomaly-samples-nupic']
df[table_2_features]

,dataset,anomaly-avg-cu,anomaly-avg-rm,anomaly-avg-tm,anomaly-avg-nupic,anomaly-prob-avg-cu,anomaly-prob-avg-rm,anomaly-prob-avg-tm,anomaly-prob-avg-nupic,anomaly-samples-cu,anomaly-samples-rm,anomaly-samples-tm,anomaly-samples-nupic
0,monthly_sp500_pca.csv,0.568590,0.807748,0.585449,0.585449,0.391225,0.391225,0.391225,0.391225,1641,1641,1641,1641
1,weekly_dow_jones.csv,0.933315,0.984454,0.935132,0.935132,0.519481,0.519481,0.519481,0.519481,2079,2079,2079,2079
2,weekly_nasdaq.csv,0.754400,0.906177,0.759195,0.759235,0.519712,0.519712,0.519712,0.519712,2080,2080,2080,2080
3,weekly_sp500.csv,0.613761,0.843591,0.621082,0.621109,0.519942,0.519942,0.519942,0.519942,2081,2081,2081,2081
4,monthly_vix_close.csv,0.047414,0.327473,0.054763,0.055010,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
5,monthly_vix_high.csv,0.063995,0.356163,0.078793,0.079040,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
6,monthly_vix_low.csv,0.031557,0.298951,0.036578,0.036824,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
7,monthly_vix_open.csv,0.055960,0.355967,0.068316,0.068532,0.753333,0.753333,0.753333,0.753333,4050,4050,4050,4050
8,daily_natural_gas.csv,0.005426,0.065698,0.008657,0.008830,0.827699,0.827699,0.827699,0.827699,5798,5798,5798,5798
9,daily_oil_prices.csv,0.016821,0.210119,0.019428,0.019533,0.879639,0.879639,0.879639,0.879639,8300,8300,8300,8300


In [10]:
table_3_features = ['dataset','infer-time-cu','infer-time-rm','infer-time-tm','total-infer-time-cu','total-infer-time-tm','total-infer-time-rm']
df[table_3_features]

,dataset,infer-time-cu,infer-time-rm,infer-time-tm,total-infer-time-cu,total-infer-time-tm,total-infer-time-rm
0,monthly_sp500_pca.csv,0.000328,0.000011,0.000355,0.538501,0.582922,0.017851
1,weekly_dow_jones.csv,0.000321,0.000009,0.000325,0.667622,0.676158,0.018905
2,weekly_nasdaq.csv,0.000374,0.000010,0.000391,0.778713,0.812974,0.021155
3,weekly_sp500.csv,0.000321,0.000010,0.000341,0.667106,0.709824,0.020715
4,monthly_vix_close.csv,0.000333,0.000011,0.000348,1.348598,1.409525,0.046240
5,monthly_vix_high.csv,0.000302,0.000011,0.000324,1.222909,1.311114,0.044332
6,monthly_vix_low.csv,0.000170,0.000009,0.000175,0.689682,0.709778,0.037166
7,monthly_vix_open.csv,0.000314,0.000011,0.000333,1.273704,1.347973,0.045857
8,daily_natural_gas.csv,0.000060,0.000007,0.000061,0.348562,0.354826,0.041731
9,daily_oil_prices.csv,0.000114,0.000008,0.000116,0.943705,0.961529,0.070303


In [12]:
df['cu-infer-speedup'] = 1 - (df['total-infer-time-cu'] / df['total-infer-time-tm'])
df['cu-accuracy-improvement'] = df['accuracy-cu'] - df['accuracy-tm']
df['cu-tm-count'] = df['anomaly-samples-cu'] - df['cu-rm-count']

table_4_features = ['dataset','cu-accuracy-improvement','cu-infer-speedup','cu-rm-count','cu-tm-count','anomaly-samples-cu','infer-time-rm','infer-time-tm']
df[table_4_features]

,dataset,cu-accuracy-improvement,cu-infer-speedup,cu-rm-count,cu-tm-count,anomaly-samples-cu,infer-time-rm,infer-time-tm
0,monthly_sp500_pca.csv,0.024985,0.076203,129,1512,1641,0.000011,0.000355
1,weekly_dow_jones.csv,0.002405,0.012624,27,2052,2079,0.000009,0.000325
2,weekly_nasdaq.csv,0.005288,0.042143,90,1990,2080,0.000010,0.000391
3,weekly_sp500.csv,0.005766,0.060180,129,1952,2081,0.000010,0.000341
4,monthly_vix_close.csv,0.006420,0.043225,181,3869,4050,0.000011,0.000348
5,monthly_vix_high.csv,0.015062,0.067275,282,3768,4050,0.000011,0.000324
6,monthly_vix_low.csv,0.002716,0.028312,121,3929,4050,0.000009,0.000175
7,monthly_vix_open.csv,0.010617,0.055097,231,3819,4050,0.000011,0.000333
8,daily_natural_gas.csv,0.000862,0.017654,116,5682,5798,0.000007,0.000061
9,daily_oil_prices.csv,0.000964,0.018538,166,8134,8300,0.000008,0.000116


In [13]:
table_5_features = ['dataset'] + sorted(list(set(df.columns) - (set(table_1_features) | set(table_2_features) | set(table_3_features)  | set(table_4_features))))
df[table_5_features]

,dataset,auc-cu,auc-rm,auc-tm,fscore-cu,fscore-rm,fscore-tm,precision-cu,precision-rm,precision-tm,recall-cu,recall-rm,recall-tm,support-cu,support-rm,support-tm
0,monthly_sp500_pca.csv,0.717988,0.584451,0.705488,0.419769,0.162399,0.395516,0.413236,0.160655,0.389287,0.436319,0.169409,0.411335,None,None,None
1,weekly_dow_jones.csv,0.518287,0.504090,0.517084,0.033671,0.007889,0.032148,0.032308,0.007817,0.031185,0.037037,0.008658,0.034632,None,None,None
2,weekly_nasdaq.csv,0.610871,0.536797,0.608225,0.210380,0.070620,0.204700,0.205626,0.069548,0.199881,0.222115,0.074038,0.216827,None,None,None
3,weekly_sp500.csv,0.681010,0.568510,0.678125,0.345960,0.133222,0.340674,0.339102,0.131628,0.333976,0.362326,0.137434,0.356559,None,None,None
4,monthly_vix_close.csv,0.988022,0.871573,0.984811,0.973580,0.739142,0.966529,0.972737,0.738508,0.965412,0.976049,0.743210,0.969630,None,None,None
5,monthly_vix_high.csv,0.980119,0.839714,0.972586,0.956473,0.676182,0.941379,0.955286,0.675694,0.940037,0.960247,0.679506,0.945185,None,None,None
6,monthly_vix_low.csv,0.989627,0.884416,0.988269,0.977335,0.765951,0.974461,0.976630,0.765324,0.973623,0.979259,0.768889,0.976543,None,None,None
7,monthly_vix_open.csv,0.980612,0.853668,0.975303,0.957708,0.703234,0.946507,0.956434,0.702472,0.945137,0.961235,0.707407,0.950617,None,None,None
8,daily_natural_gas.csv,0.998189,0.977833,0.997757,0.995803,0.954710,0.994972,0.995579,0.954497,0.994786,0.996378,0.955674,0.995516,None,None,None
9,daily_oil_prices.csv,0.997470,0.960839,0.996988,0.993884,0.918908,0.992878,0.993404,0.918216,0.992405,0.994940,0.921687,0.993976,None,None,None
